In [1]:
from dotenv import load_dotenv
load_dotenv()

from openai import OpenAI
client = OpenAI()


In [5]:
import pandas as pd
file_path = 'spotify_songs.csv'
sales_data = pd.read_csv(file_path, nrows=20, encoding='latin1')

sales_data.head()


,track_id,track_name,track_artist,track_popularity,track_album_id,track_album_name,track_album_release_date,playlist_name,playlist_id,playlist_genre,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,6f807x0ima9a1j3VPbc7VN,I Don't Care (with Justin Bieber) - Loud Luxur...,Ed Sheeran,66,2oCs0DGTsRO98Gh5ZSl2Cx,I Don't Care (with Justin Bieber) [Loud Luxury...,2019-06-14,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,6,-2.634,1,0.0583,0.1020,0.000000,0.0653,0.518,122.036,194754
1,0r7CVbZTWZgbTCYdfa2P31,Memories - Dillon Francis Remix,Maroon 5,67,63rPSO264uRjW1X5E6cWv6,Memories (Dillon Francis Remix),2019-12-13,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,11,-4.969,1,0.0373,0.0724,0.004210,0.3570,0.693,99.972,162600
2,1z1Hg7Vb0AhHDiEmnDE79l,All the Time - Don Diablo Remix,Zara Larsson,70,1HoSmj2eLcsrR0vE9gThr4,All the Time (Don Diablo Remix),2019-07-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-3.432,0,0.0742,0.0794,0.000023,0.1100,0.613,124.008,176616
3,75FpbthrwQmzHlBJLuGdC7,Call You Mine - Keanu Silva Remix,The Chainsmokers,60,1nqYsOef1yKKuGOVchbsk6,Call You Mine - The Remixes,2019-07-19,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,7,-3.778,1,0.1020,0.0287,0.000009,0.2040,0.277,121.956,169093
4,1e8PAfcKUYoKkxPhrHqw4x,Someone You Loved - Future Humans Remix,Lewis Capaldi,69,7m7vv9wlQ4i0LFuJiE2zsQ,Someone You Loved (Future Humans Remix),2019-03-05,Pop Remix,37i9dQZF1DXcZDD7cfEKhW,pop,...,1,-4.672,1,0.0359,0.0803,0.000000,0.0833,0.725,123.976,189052


In [14]:
# Create assistant and conversation

file = client.files.create(
    file=open(file_path, "rb"),
    purpose='assistants',
)

assistant = client.beta.assistants.create(
    instructions="As a Data Science Assistant, given data and a query, you can write appropriate code and create appropriate visualizations.",
    model="gpt-4-0125-preview",
    tools=[
        {"type": "code_interpreter"}
    ],
    tool_resources={
        "code_interpreter": {
            "file_ids": [file.id]  # Here we add the file id
        }
    }
)

print(assistant)


Assistant(id='asst_Jjy1hqoTqMfl5ETvsNsyvnAp', created_at=1748895513, description=None, instructions='As a Data Science Assistant, given data and a query, you can write appropriate code and create appropriate visualizations.', metadata={}, model='gpt-4-0125-preview', name=None, object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=['file-7YtGXruJzD2kWp5B53hVFq']), file_search=None), top_p=1.0, reasoning_effort=None)


In [41]:
# **thread** can start a conversation

thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "Based on the contents of the dataset, select the most representative fields, create multiple visualization charts showing the characteristics and trends of best-selling songs over the years (by release date), making sure each musician (artist) is represented by a different color.",
            "attachments": [
                {
                    "file_id": file.id,
                    "tools": [
                        { "type": "code_interpreter" }
                    ]
                }
            ]
        }
    ]
)
print(thread)


Thread(id='thread_S5CSD5XWV9fda27inNNT5z7F', created_at=1748897348, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=['file-7YtGXruJzD2kWp5B53hVFq']), file_search=None))


In [42]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
print(run)


Run(id='run_Rpfa5mRzp1J0IHw3FoOCMuLG', assistant_id='asst_Jjy1hqoTqMfl5ETvsNsyvnAp', cancelled_at=None, completed_at=None, created_at=1748897351, expires_at=1748897951, failed_at=None, incomplete_details=None, instructions='As a Data Science Assistant, given data and a query, you can write appropriate code and create appropriate visualizations.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-0125-preview', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_S5CSD5XWV9fda27inNNT5z7F', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={}, reasoning_effort=None)


In [43]:
import time
while True:
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    try:
        messages.data[0].content[0].image_file
        time.sleep(5)
        print('Chart created!')
        if messages.data and messages.data[0].content:
            print('Current Message:', messages.data[0].content[0])
        break
    except:
        time.sleep(10)
        print('Your Assistant is working on creating the chart...')
        if messages.data and messages.data[0].content:
            print('Current Message:', messages.data[0].content[0])


Your Assistant is working on creating the chart...
Current Message: TextContentBlock(text=Text(annotations=[], value='Based on the contents of the dataset, select the most representative fields, create multiple visualization charts showing the characteristics and trends of best-selling songs over the years (by release date), making sure each musician (artist) is represented by a different color.'), type='text')
Your Assistant is working on creating the chart...
Current Message: TextContentBlock(text=Text(annotations=[], value="To assist you effectively, I'll start by inspecting the contents of the uploaded file to understand its structure and determine which fields are most relevant for analyzing trends and characteristics of best-selling songs. Once this is clear, I can proceed to select the most representative fields and create the visualizations you asked for. Let's begin by checking the file."), type='text')
Your Assistant is working on creating the chart...
Your Assistant is worki

In [44]:
def convert_file_to_png(file_id, write_path):
    data = client.files.content(file_id)
    data_bytes = data.read()
    with open(write_path, "wb") as file:
        file.write(data_bytes)

plot_file_id = messages.data[0].content[0].image_file.file_id
image_path = "music trend analysis.png"
convert_file_to_png(plot_file_id, image_path)

plot_file = client.files.create(
    file=open(image_path, "rb"),
    purpose='assistants'
)


In [25]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
[message.content[0] for message in messages.data]


[ImageFileContentBlock(image_file=ImageFile(file_id='file-WNqtyy2CJsMeGAGbcZ3FjU', detail=None), type='image_file'),
 TextContentBlock(text=Text(annotations=[], value="The dataset includes fields that describe various aspects of the songs, such as `track_name`, `track_artist`, `track_popularity`, `track_album_release_date`, along with detailed audio features like `acousticness`, `danceability`, `energy`, `tempo`, and more.\n\nTo create a visualization showcasing the trends and characteristics of best-selling songs over the years, we can use the following fields:\n- `track_name`\n- `track_artist`\n- `track_album_release_date`\n- `track_popularity`\n\nThese fields can help us understand the popularity trends of songs over time. For the visualization, we could consider plotting `track_popularity` over the `track_album_release_date`, with each `track_artist` represented by a different color.\n\nLet's proceed by summarizing the data based on these fields to prepare for the visualization. We

In [45]:
# AI-generated data insights

def submit_message_wait_completion(assistant_id, thread, user_message, file_ids=None):
    for run in client.beta.threads.runs.list(thread_id=thread.id).data:
        if run.status == "in_progress":
            print(f"Waiting for Run {run.id} to complete...")
            while True:
                run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id).status
                if run_status in ['succeeded', 'failed']:
                    break
                time.sleep(5)

    params = {
        'thread_id': thread.id,
        'role': 'user',
        'content': user_message,
    }

    if file_ids:
        attachments = [{"file_id": file_id, "tools": [ { "type": "code_interpreter" } ]} for file_id in file_ids]
        params['attachments'] = attachments

    client.beta.threads.messages.create(**params)

    run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant_id)
    return run


In [52]:
submit_message_wait_completion(
    assistant.id,
    thread,
    "Based on the chart you just created, give me two sentences of about 20 words each, describing the most important insights. This will be used in a slide presentation to reveal the 'secrets' behind the data."
)


Run(id='run_z118oOtFTffbW3lc6FDv8aYd', assistant_id='asst_Jjy1hqoTqMfl5ETvsNsyvnAp', cancelled_at=None, completed_at=None, created_at=1748898178, expires_at=1748898778, failed_at=None, incomplete_details=None, instructions='As a Data Science Assistant, given data and a query, you can write appropriate code and create appropriate visualizations.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-0125-preview', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_S5CSD5XWV9fda27inNNT5z7F', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={}, reasoning_effort=None)

In [53]:
def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id)

time.sleep(10)
response = get_response(thread)
bullet_points = response.data[0].content[0].text.value
print(bullet_points)


The visualization unveils a diverse trajectory of artist popularity, where certain years mark remarkable peaks, likely tied to hit singles or albums. A discernible trend of increasing track popularity post-2015 across multiple artists suggests evolving listener preferences or effective promotional strategies.


In [54]:
submit_message_wait_completion(
    assistant.id,
    thread,
    "Based on the plots and key points you created, come up with a very short title for the slide. It should only reflect the main insights you have drawn."
)


Run(id='run_9nbDronXrCoUDcw7NIj9Qb76', assistant_id='asst_Jjy1hqoTqMfl5ETvsNsyvnAp', cancelled_at=None, completed_at=None, created_at=1748898303, expires_at=1748898903, failed_at=None, incomplete_details=None, instructions='As a Data Science Assistant, given data and a query, you can write appropriate code and create appropriate visualizations.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-0125-preview', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_S5CSD5XWV9fda27inNNT5z7F', tool_choice='auto', tools=[CodeInterpreterTool(type='code_interpreter')], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={}, reasoning_effort=None)

In [55]:
time.sleep(10)
response = get_response(thread)
title = response.data[0].content[0].text.value
print(title)


"Evolving Popularity: Peaks & Trends in Top Artists' Music (2015-2019)"


In [56]:
company_summary = "We are a CD wholesaler, but our chairman is also a pop singer!"

response = client.images.generate(
    model="dall-e-3",
    prompt=f"Based on this company summary {company_summary}, create an inspirational photo showcasing growth and the path forward. This will be used for the quarterly sales planning meeting.",
    size="1024x1024",
    quality="hd",
    n=1
)

image_url = response.data[0].url

import requests
dalle_img_path = 'Music Trend Chart.png'
img = requests.get(image_url)

with open(dalle_img_path, 'wb') as file:
    file.write(img.content)

dalle_file = client.files.create(
    file=open(dalle_img_path, "rb"),
    purpose='assistants'
)


In [58]:
# Create the final PPT

title_template = """


from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.dml.color import RGBColor

prs = Presentation()

blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)

background = slide.background
fill = background.fill
fill.solid()
fill.fore_color.rgb = RGBColor(0, 0, 0)

left = Inches(0)
top = Inches(0)
height = prs.slide_height
width = prs.slide_width * 3 / 5
pic = slide.shapes.add_picture(image_path, left, top, width=width, height=height)

left = prs.slide_width * 3 / 5
top = Inches(2)
width = prs.slide_width * 2 / 5
height = Inches(1)
title_box = slide.shapes.add_textbox(left, top, width, height)
title_frame = title_box.text_frame
title_p = title_frame.add_paragraph()
subtitle_p.text = subtitle_text
subtitle_p.font.size = Pt(38)
subtitle_p.font.color.rgb = RGBColor(255, 255, 255)
subtitle_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER

"""

data_vis_template = """


from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_PARAGRAPH_ALIGNMENT
from pptx.dml.color import RGBColor

prs = Presentation()

blank_slide_layout = prs.slide_layouts[6]
slide = prs.slides.add_slide(blank_slide_layout)

background = slide.background
fill = background.fill
fill.solid()
fill.fore_color.rgb = RGBColor(0, 0, 0)

image_path = data_vis_img
title_text = "Maximizing Profits: The Dominance of Online Sales and Direct Sales Optimization"
bullet_points = "• Online sales consistently lead in profitability across quarters, indicating a strong digital market presence.\n• Direct sales show fluctuations, indicating variability in performance and the need for targeted improvements."

left = Inches(0.2)
top = Inches(1.8)
height = prs.slide_height - Inches(3)
width = prs.slide_width * 3 / 5
pic = slide.shapes.add_picture(image_path, left, top, width=width, height=height)

left = Inches(0)
top = Inches(0)
width = prs.slide_width
height = Inches(1)
title_box = slide.shapes.add_textbox(left, top, width, height)
title_frame = title_box.text_frame
title_frame.margin_top = Inches(0.1)
title_p = title_frame.add_paragraph()
title_p.text = title_text
title_p.font.bold = True
title_p.font.size = Pt(28)
title_p.font.color.rgb = RGBColor(255, 255, 255)
title_p.alignment = PP_PARAGRAPH_ALIGNMENT.CENTER

left = prs.slide_width * 2 / 3
top = Inches(1.5)
width = prs.slide_width * 1 / 3
height = Inches(4.5)
insights_box = slide.shapes.add_textbox(left, top, width, height)
insights_frame = insights_box.text_frame
insights_p = insights_frame.add_paragraph()
insights_p.text = "Key Insights:"
insights_p.font.bold = True
insights_p.font.size = Pt(24)
insights_p.font.color.rgb = RGBColor(0, 128, 100)
insights_p.alignment = PP_PARAGRAPH_ALIGNMENT.LEFT
insights_frame.add_paragraph()

bullet_p = insights_frame.add_paragraph()
bullet_p.text = bullet_points
bullet_p.font.size = Pt(12)
bullet_p.font.color.rgb = RGBColor(255, 255, 255)
bullet_p.line_spacing = 1.5
"""

title_text = "Birdsong Paradise"
subtitle_text = "2025 Sales Conference"


submit_message_wait_completion(
    assistant.id,
    thread,
    f"""Using the provided code template, create a PPTX slide that conforms to the template format, but use the image, company name/title, and file name/subtitle included in this message.
    Use the image file included in this message as the image_path image on this first slide and use the company name {title_text} as the title_text variable,
    and use the subtitle text {subtitle_text} as the subtitle_text variable.

    Then, using the following code template, create a second slide: {data_vis_template}, creating a PPTX slide that conforms to the template format but uses the company name/title and file name/subtitle:
    {data_vis_template}. Important note: Use the second attached image (line chart) you created earlier in this thread as the data_vis_img image, and use the data visualization title you created earlier as the title_text variable,
    and use the insights bullet points list you created earlier as the bullet_points variable. Output these two slides as a .pptx file. Ensure the output consists of two slides, each conforming to the respective template provided in the prompt.""",
    file_ids=[dalle_file.id, plot_file.id]
)

while True:
    try:
        response = get_response(thread)
        pptx_id = response.data[0].content[0].text.annotations[0].file_path.file_id
        print("Successfully retrieved pptx_id:", pptx_id)
        break
    except Exception as e:
        print("Your Assistant is working hard on creating the slides...")
        time.sleep(10)


import io
pptx_id = response.data[0].content[0].text.annotations[0].file_path.file_id
ppt_file = client.files.content(pptx_id)
file_obj = io.BytesIO(ppt_file.read())
with open("Promotional_Conference.pptx", "wb") as f:
    f.write(file_obj.getbuffer())




Your Assistant is working hard on creating the slides...
Your Assistant is working hard on creating the slides...
Your Assistant is working hard on creating the slides...
Your Assistant is working hard on creating the slides...
Your Assistant is working hard on creating the slides...
Your Assistant is working hard on creating the slides...
Your Assistant is working hard on creating the slides...
Successfully retrieved pptx_id: file-HN34f5fYMWKaY8SjWg6a9N
